In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os
import re # Regex 
from publication_plotstyle import *

# The RBC class is functional. 

- Verified that many of the things work from the class. 
- But the steady state and time is a mess
- It's still not the best code
- Hooray! The RBC class is ready to use! 


## Functions to be used everywhere

## The Parameters Class 

- The class has many arguments. All of them need to be given

- The functions are all called implicitly. So, all parameters need to be adjusted as soon as class initializing.

- The initialization __init__ returns the object. This is necessary for RBC class. 


### Attributes 

1. U_fall, T_fall
1. Z, dz
1. N_nz
1. nruns, myruns

1. get_runs()
1. getZ()
1. get_free_fall_units()


In [2]:
class Params:
    
    """ This class contains mesh, geometry and all such params """
    
    #======================================
    
    def __init__(self, 
                 Ra, Pr, Gamma, 
                 nelz, N, 
                 visc, conduc, 
                 directory):
        
        print('==== Set Parameters of Study ==== ')
        print()
        
        # RBC 
        self.Ra = Ra 
        self.Pr = Pr 
        self.Gamma = Gamma 
        
        # geometry and meshing parameters:
        self.geo_mesh = {
        'nelz' : nelz,
        'N' : N,
        'Gamma' : Gamma,
        'D' : 1.0,
        'H' : 1.0,
        }

        self.geo_mesh['N_nz'] = self.geo_mesh['N'] * self.geo_mesh['nelz']
        
        # Physics:
        self.visc = visc
        self.conductivity = conduc
        self.deltaT = 1.0 
        
        # Frequently used stuff:
        self.N_nz = self.geo_mesh['N_nz']
        self.dir = directory
                
        # Functions to get other stuff:
        self.get_runs()
        self.get_free_fall_units()
        self.getZ()
        
        return self
        
    # ==================

    def get_runs(self):
        
        self.myruns = [r for r in sorted(os.listdir(self.dir)) if r.startswith("RUN")]
        self.nruns = np.arange(1,len(self.myruns)+1,dtype = int)
        print(*self.myruns)  #======> works
        print()
        
    # ===========================
    
    def get_free_fall_units (self):
        print('Viscosity = ', self.visc)
        
        self.U_fall = self.visc * np.sqrt(self.Ra/self.Pr) # free fall velocity
        self.T_fall = self.geo_mesh['H']/self.U_fall
        print(f'Free Fall Velocity = {self.U_fall} \t Free Fall Time = {self.T_fall}')
        print()
        
    # ===========================
    
    def getZ(self):
        
        ' Get Z and dz from any vertical mean data file '
        
        self.Z = np.loadtxt(self.dir + 'RUN1/ver_temp.dat',unpack = True)[0,0:self.N_nz]
        self.Z = self.Z/self.geo_mesh['H']
        self.dz = self.Z[1:] - self.Z[:-1]
        
# -----------------------------

## The Timing class 

### Attributes
1. init()
    1. iostep, endstep 
1. add_runs() - to add data to iostep, endstep for newest runs
1. get_iostep() - set the value of iotimesteps : can be a property class 
    1. iotimesteps
1. stdout() 
1. IO_times() -- IMP 
    1. iotimes - all the IO times for writing vert means
    1. dt_at_iostep
    2. cour_at_iostep
1. get_steady_states() - get steady states array after assigning a stating point. 
1. get_stdTime(run, timestep) - get the time at any given timestep

In [3]:
class Time: 
    
    """ Time is an important variable in RBC. Lots of time variables are there """
    
    def __init__(self, iostep, endstep, steady_start, directory):
        
        self.dir = directory
        self.iostep = iostep
        self.endstep = endstep # might be useful
        
        print('==== Get Time information ====')
        print()
        
        # Functions:
        self.get_iostep(iostep, endstep)
        self.IO_times()
        
        _ = self.get_steady_states(steady_start)
        
    # ===============================
    # might be useful
    
    def add_runs(self, iostep, endstep):
        
        'Append the iostep and endstep for one or more runs'
        
        iostep = np.array(iostep)
        endstep = np.array(endstep)
        
        self.iostep = np.concat(self.iostep, iostep)
        self.endstep = np.concat(self.endstep, endstep)
              
    # ===============================

    def get_iostep(self, iostep, endstep):
        self.iotimesteps = {i+1:np.arange(io, end+1, io, dtype = int) 
                            for i, (io,end) in enumerate(zip(iostep, endstep) )
                           }
        
    # ===============================
    
    def get_stdoutTime(self, run, timestep):
        
        ' Gets time data for any run and any timestep from the files '
        
        delimiters = "Step","DT=","C="," ",",","\n","\t","t=" 
        regex_pattern = '|'.join(map(re.escape,delimiters))
        
        with open(self.dir + 'RUN' + ['','_'][int(run>=10)] 
                             + str(run) +'/DT.dat' ) as DTFile:
            
            line = DTFile.readlines()[timestep - 1]
            each_line = [float(elem) for elem in re.split(regex_pattern, line) 
                         if elem !=''][:-2]
            
            time = each_line[1]
            DT = each_line[2]
            cour = each_line[3]
            
        return time, DT, cour
        
    # ==============================
    
    def IO_times(self):
        
        ''' removes using RUN for any data, getting at all iosteps '''
    
        self.iotimes = np.array([])        
        self.dt_at_iostep = np.array([])
        self.cour_at_iostep = np.array([])
        
        for run, io_per_run in self.iotimesteps.items():
            for r in io_per_run:
                
                s, d, c = self.get_stdoutTime(run, r)
                self.iotimes = np.append(self.iotimes,s)
                self.dt_at_iostep = np.append(self.dt_at_iostep, d)
                self.cour_at_iostep = np.append(self.cour_at_iostep, c)
       
    # =======================================

    def get_steady_states(self, steady_start):
        
        ' Get the steady state times. Pretty handy sometimes '
        
        self.steady_states = np.array(
                        list(filter(lambda x : x >= steady_start, self.iotimes))
                    )
        print('steady state times are in between : \n', self.steady_states[0], 
                                                          self.steady_states[-1]) 
        print()
        
        return self.steady_states
    
    # =======================================
    

#----------------------

****

## Nusselt Number class

- Composition of Params and Time classes. 
- Overloading get_Nusselt. One of them is called de-facto
- Large arrays here. 3 * all the timesteps from all the run put here in Nusselt number, is there a way around it?
- getting Nu for top and bottom plates, and all the times written out. Should have done this in the Time class, but is more efficient here. 

### Attributes 

1. get_Nusselt - for any run, for any timestep, __returns__ bot and top Nu at the time. 
1. get_Nusselt - for all runs, for all timesteps 
    1. Nu_top_stdout 
    1. Nu_bot_stdout
    1. all_times 
    
    
1. get_IO_Nusselt - for finding out Nusselt numbers from stdout at all io times.
    1. Nu_top_io
    1. Nu_bot_io 
    
    
1. Nu_ensemble_avg - calculate the ensemble average at all given times from steady state. 
1. Nu_VolAvg_time - calculate the volume average at any, all time. 


In [4]:
class Nusselt:
    
    ''' Gets Nusselt Number data from stdout, and calculation routines '''    
    
    def __init__(self, params, time, 
                 ver_uzte, ver_dtdz, 
                 steady_start):
        
        print('==== Nusselt Number Collect ====')
        print()

        print('Steady Start time = ', steady_start)
        
        self.get_Nusselt(params, time)
        self.get_IO_Nusselt(params, time) 
        
        # for Nusselt_where():
        self.params = params
        self.time = time 
        self.ver_uzte = ver_uzte 
        self.ver_dtdz = ver_dtdz
        
        # ensemble average 
        self.Nu_ensemble_avg = 1 + np.sqrt(params.Ra * params.Pr) * ver_uzte.ensemble_avg( 
                                                         steady_start)
        
        # volume avg. as a function of time 
        self.Nu_VolAvg_time = 1 + np.sqrt(params.Ra * params.Pr) * ver_uzte.vol_avg_in_time( 
                                                           steady_start)
                                                           
        print('Ensemble averaged Nusselt Number = ', self.Nu_ensemble_avg)
        print()
        
    # =====================================
    
    def get_Nusselt(self, run, timestep):
        
        ' Get Nu for top and bottom plate for a given run and timestep'
        
        Nufile = open(self.dir + 'RUN' + ['','_'][int(run>=10)] + str(run) + 
                    '/Nusselt.dat'
                   )
        
        Nu_bot, Nu_top = np.loadtxt(Nufile, 
                                    usecols = [0,*range(2,6)]) [timestep - 1][-2:]                         
        return Nu_bot, Nu_top
    
    # =====================================
    
    def get_Nusselt(self, params, time):
 
        ' Gets Nusselt for top and bottom plates for all times '
           
        self.Nu_top_stdout = np.array([])
        self.Nu_bot_stdout = np.array([])
        time.all_times = np.array([])
        
        for nr, run in zip(params.nruns, params.myruns):

            Nu_data = np.loadtxt(params.dir+run+'/Nusselt.dat',
                                               usecols=[0,*range(2,6)],
                                               unpack = True)
            
            # store all the times for which Nu exists
            # add time_all to time obj .. 
            time.all_times = np.append(time.all_times, Nu_data[1])
            
            # Nusselt at top plate for all times
            self.Nu_top_stdout = np.append(self.Nu_top_stdout, Nu_data[-1])

            # Nusselt at bottom plate for all times
            self.Nu_bot_stdout = np.append(self.Nu_bot_stdout, Nu_data[-2])

        self.Nu_top_stdout.setflags(write = False)
        self.Nu_bot_stdout.setflags(write = False)
        
        print("Obtained all Nusselt Number data from stdout")
        
    # ==============================================
    
    def get_IO_Nusselt(self, params, time):
        
        self.Nu_top_io = np.array([])
        self.Nu_bot_io = np.array([])
        
        for run, my_run in zip(params.nruns, params.myruns):
            Nu_data = np.loadtxt(params.dir + my_run 
                                 + '/Nusselt.dat',
                                 usecols = [0,*range(2,6)], 
                                 unpack = True
                                )
            
            for timestep in time.iotimesteps[run]:
                self.Nu_top_io = np.append(self.Nu_top_io,
                                           Nu_data[-1][timestep - 1]
                                          )
            
                self.Nu_bot_io = np.append(self.Nu_bot_io,
                                           Nu_data[-2][timestep - 1]
                                          )
    
        self.Nu_top_io.setflags(write = False)
        self.Nu_bot_io.setflags(write = False)
        
        print('Obtained all Nusselt Number data at IO Times')
         
    # ==============================================
    
    def Nusselt_where(self, run, timestep, z):
        
        '''
        Find out nusselt number at any given (Z/H, t/Tf) using Exact Equation
        z : 0 - 1 any value 
        '''
        
        def get_nearest_z(z):
            Δz = self.params.Z - z
            nearest_zloc = np.where(abs(Δz) == min(abs(Δz)) )[0][0] + 1
            return nearest_zloc
    
        uzt = self.ver_uzte.get_Vert(run, timestep)[get_nearest_z(z) - 1]
        dtdz = self.ver_dtdz.get_Vert(run, timestep)[get_nearest_z(z) - 1]
        
        Nu_z = np.sqrt(self.params.Ra * self.params.Pr ) * uzt - dtdz
        return Nu_z
    
    # ===============================================

***

## Vertical means:

- Vertical means is a class, with all types of vertical means as objects of that class. 
- All of them share the same character underneath.
- Composite class; uses 1 or 2 variables from the previous 2 classes. Hence, a composition is apt


- which_var : Tells which variable is being considered. A sanity check tool 
- get_Vert() : an efficient way to get what is needed only. No need to store large arrays. 
- some data is 'leaked', or reassigned to different objects of composite class; to ensure readability.

- Unable to integrate ver_rms into the default read functions. 


In [5]:
class Vertical_Means:
    
    ' Get the vertical mean profiles. Only take what you need '

    def __init__(self, params, time):
        
        # get all the required params from the objects. More efficient
        self.dir = params.dir
        self.N_nz = params.N_nz
        
        self.iotimesteps = time.iotimesteps
        self.endstep = time.endstep
        self.iostep = time.iostep
        
        self.params = params
        self.time = time
        
        
    def which_var(self, var):
        self.var = var
        
    # ==========================================
    
    def get_Vert(self, run, timestep):
        
        '''
        Get the vertical mean for a given run and timestep 
        See if the file exists. 
        Timestep --> iotimestep 
        '''
        
        try:
            file = open(self.dir + 'RUN' + ["","_"][int(run>=10)] 
                   + str(run) + '/ver_' + self.var + '.dat')
            
        except: # put the vertical mean as nan if the file does not exist
            
            vert = [np.nan] * self.N_nz
            # print(f'{self.var} does not exist for this run {run}. Setting to NaN')
        
        else:
            
            try:
                iloc = np.where(self.iotimesteps[run] == timestep)[0][0] + 1
            except:
                print('timestep not written, exit')
                return
            else:
                N_nz = self.N_nz
                vert = np.loadtxt(file, unpack=True)[1, (iloc - 1)*N_nz : iloc*N_nz]
        
        return vert
    
    # =========================================
    def vol_avg_in_time(self, start): 
        
        ' Returns spatial avg. as a function of time after a given time '
        
        self.vert_avg = np.array([])
        
        for run in self.params.nruns:
            for timestep in self.time.iotimesteps[run]:
                
                time_at_io = self.time.get_stdoutTime(run,timestep)[0]
                if time_at_io >= start:
                   
                    # get data for run and time 
                    u = self.get_Vert(run, timestep) 
                    
                    # if np.nan then continue 
                    if np.nan in u: 
                        continue
                    
                    # get integral and average 
                    vert_sum = 1./self.params.geo_mesh['H'] * (
                                    np.sum(self.params.dz * 0.5 * (u[1:] + u[:-1]) )
                                                            )
                                
                    self.vert_avg = np.append(self.vert_avg, vert_sum)
                
        del vert_sum, u
        return self.vert_avg          
        
    # ==========================================
    
    def ensemble_avg(self, start):
        
        steady_states = self.time.get_steady_states(steady_start = start)
        δt = steady_states[1:] - steady_states[:-1]
        
        space_avg = self.vol_avg_in_time(start)      
        self.esb_avg = np.sum( δt * 0.5 * (space_avg[1:] + space_avg[:-1]) )        
        self.esb_avg = self.esb_avg/(steady_states[-1] - steady_states[0])
    
        del space_avg 
        return self.esb_avg
    
    # =========================================
    
    def TimeAvg_in_z(self, start_time, stop_time):
        
        ''' bit hard function to do. Referred from Moving average fn() '''
    
        '''  
        nruns --> which set of runs: if we use ver_rms, we need rms_runs 
        start_time, stop_time --> in units of free fall time 
        '''
        
        start_time = start_time * self.params.T_fall
        stop_time = stop_time * self.params.T_fall
    
        mov_avg = np.zeros(self.params.N_nz)
        δt_sum = 0
    
        for run in self.params.nruns:
            for tloc, timestep in enumerate(self.time.iotimesteps[run]):
            
                iot = self.time.get_stdoutTime(run, timestep)[0]
                v = self.get_Vert(run, timestep)

                if iot < start_time or iot > stop_time:
                    continue 
            
                prev_iot = 0.0
                prev_v = np.zeros_like(v)
            
                if tloc == 0 and run == self.params.nruns[0]:
                    prev_v = v
                    prev_iot = iot - 1e-6 # just so that trapez rule becomes easy. 
                
                elif tloc == 0 and run != self.params.nruns[0]:
                    prev_ts = self.time.iotimesteps[run-1][-1]
                    prev_v = self.get_Vert(run-1,prev_ts)
                    prev_iot = self.time.get_stdoutTime(run-1, prev_ts)[0]
                
                elif tloc!=0:
                    prev_ts = self.time.iotimesteps[run][tloc - 1]
                    prev_v = self.get_Vert(run, prev_ts)
                    prev_iot = self.time.get_stdoutTime(run, prev_ts)[0]
        
                δt = iot - prev_iot ; δt_sum += δt
                mov_avg += δt * 0.5 *(v + prev_v)
    
        mov_avg = 1./δt_sum * mov_avg
        return mov_avg
    
    #==========================================


## The RBC Class 

- Lots of arguments: due to the params class being inherited. 
- Time class not inherited. Can be, so that all the time variables can be easily accessed here 
- Using classes as structs: Look at class kolmogorov. 
- The steady_state concept is very bad. 

In [6]:
class RBC(Params):
    
    ''' 
        THE RBC MASTERHEAD Class Composition 
        Input the iostep and endstep arrays
    '''
    
    def __init__(self, Ra, Pr, Gamma, 
                 nelz, N,
                 visc, conduc,
                 iostep, endstep, 
                 steady_start,
                 directory):
        
        self.params = super(RBC, self).__init__(Ra, Pr, Gamma,
                                  nelz, N,
                                  visc, conduc, 
                                  directory)

        self.time = Time(iostep, endstep,
                         steady_start,
                         directory)

        
        print('==== Get Vertical_Means Objects ====')
        print()
        
        vert_files = 'temp','dtdz','epst','epsv','uzte','rmsvh','rms' 
        
        for vert_file in vert_files:
            exec( f'self.ver_{vert_file}' + 
                    ' = Vertical_Means(self.params, self.time)'
                )
            exec( f'self.ver_{vert_file}' + '.which_var(  "' + vert_file + '" ) '
                )
             
        # set nusselt numbers
        # steady_start value as defined in the RBC, time._init()_ 
        self.nusselt = Nusselt(self.params, self.time, 
                               self.ver_uzte, self.ver_dtdz, 
                               steady_start) 
        
        self.all_times = self.time.all_times # which is set in Nu class lol
        self.steady_start = steady_start
        
    # ============================
    
    # Kolmogorov 
    
    @property
    def τ_K(self):
        
        ' the ensemble averaged kolmogorov time scale'
        
        prefac = np.sqrt(self.Pr)/np.sqrt(self.Ra)
        epsv_ensemble_avg = self.ver_epsv.ensemble_avg(self.steady_start)
        scaling = 1.0/(epsv_ensemble_avg)**(-0.25)
        
        return prefac * scaling 
    
    # ----------------------------
    
    @property 
    def τ_K_in_time(self):
        
        ' returns tauK as a function of time - this is an array '
        
        prefac = np.sqrt(self.Pr)/np.sqrt(self.Ra) 
        epsv_vol_in_time = self.ver_epsv.vol_avg_in_time(self.steady_start)
        scaling = 1.0/(epsv_vol_in_time)**(-0.25)
        
        return prefac * scaling
        
    # ============================
    
    @property 
    def η_K(self):
        pass 
    
    # ----------------------------
    
    @property
    def η_K_in_time(self):
        pass 
    
    

- Use a dictionary to provide parameters to the RBC objects. Much cleaner. 

***
***

## Library of Functions that do not belong to the class

A set of functions that are required, but do not need to belong to any of the classes. 

- Redundant Moving Average : A simple reimann sum is used because calling a more accurate integration is not working correctly 

$$
\left < \phi \right > = \frac{ \sum \phi _i t_i  }{\sum t_i}
$$



In [ ]:
# - could add this into the rbc class 

def moving_average (rbc, vert, nruns, start_time, stop_time):
    
    '''
    rbc --> object 
    vert --> which vert 
    nruns --> which set of runs: if we use ver_rms, we need rms_runs 
    start_time, stop_time --> in units of free fall time 
    '''
    
    start_time = start_time * rbc.T_fall
    stop_time = stop_time * rbc.T_fall
    
    mov_avg = 0.0
    δt_sum = 0
    
    for run in nruns:
        for tloc, timestep in enumerate(rbc.time.iotimesteps[run]):
            
            iot = rbc.time.get_stdoutTime(run, timestep)[0]
            v = vert.get_Vert(run, timestep)

            if iot < start_time or iot > stop_time:
                continue 
            
            prev_iot = 0.0
            prev_v = np.zeros_like(v)
            
            if tloc == 0 and run == nruns[0]:
                prev_v = v
                prev_iot = iot - 1e-6 # just so that trap. rule becomes easy.  
                
            elif tloc == 0 and run!= nruns[0]:
                prev_ts = rbc.time.iotimesteps[run-1][-1]
                prev_v = vert.get_Vert(run-1,prev_ts)
                prev_iot = rbc.time.get_stdoutTime(run-1, prev_ts)[0]
                
            elif tloc!=0:
                prev_ts = rbc.time.iotimesteps[run][tloc - 1]
                prev_v = vert.get_Vert(run, prev_ts)
                prev_iot = rbc.time.get_stdoutTime(run, prev_ts)[0]
        
            δt = iot - prev_iot ; δt_sum += δt
            mov_avg += δt * 0.5 *(v + prev_v)
    
    mov_avg = 1./δt_sum * mov_avg
    return mov_avg
        
# =============================================

In [ ]:
def trap_rule(d_height, u): 
    if d_height.size != u.size : 
        print('arrays unequal'); return 
    return 0.5 * np.sum( d_height * (u[:-1] + u[1:]) )

*** 
***
